In [2]:
import re
import csv
import json
import urllib.request
import numpy as np
import pandas as pd
import folium
import pymc3 as pm
import matplotlib
import matplotlib.pylab as plt

import warnings
warnings.filterwarnings('ignore')

In [3]:
f = open('.appid')
j = json.loads(f.read())
appid = j['appid']
f.close()

In [4]:
api_version = "2.1"
base_url = "http://api.e-stat.go.jp/rest/{api_version}/app/".format(api_version=api_version)

In [5]:
# 都道府県別の自殺数

get_type="getStatsData"
stats_data_id="0003030127"
url =base_url + "{}?appId={}&statsDataId={}".format(get_type, appid, stats_data_id)
#print(url)
get_type="getSimpleStatsData"
stats_data_id="0003030127"
lv_cat_01="2"
cd_cat_02="129"
cd_cat_03="1"
section_header_flg="2"
url = base_url + "{}?appId={}&statsDataId={}&lvCat01={}&cdCat02={}&cdCat03={}&sectionHeaderFlg={}".format(get_type, appid, stats_data_id, lv_cat_01, cd_cat_02, cd_cat_03, section_header_flg)
data_pref_die = urllib.request.urlopen(url).read().decode("utf8")

In [6]:
# 都道府県別、年齢階級別の人口

get_type="getStatsData"
stats_data_id="0003014716"
url = base_url + "{}?appId={}&statsDataId={}".format(get_type, appid, stats_data_id)
#print(url)
get_type="getSimpleStatsData"
stats_data_id="0003014716"
cd_cat_01="000"
cd_cat_02_from="01001"
cd_cat_02_to="04018"
lv_area="2"
section_header_flg="2"
url = base_url + "{}?appId={}&statsDataId={}&cdCat01={}&cdCat02From={}&cdCat02To={}&lvArea={}&sectionHeaderFlg={}".format(get_type, appid, stats_data_id, cd_cat_01, cd_cat_02_from, cd_cat_02_to, lv_area, section_header_flg)
data_pref_age_pop = urllib.request.urlopen(url).read().decode("utf8")

In [7]:
# 全国、年齢階級別の自殺数

get_type="getStatsData"
stats_data_id="0003031497"
url =base_url + "{}?appId={}&statsDataId={}".format(get_type, appid, stats_data_id)
#print(url)
get_type="getSimpleStatsData"
stats_data_id="0003031497"
lv_cat_01="2"
cd_cat_02="268"
cd_cat_03="1"
section_header_flg="2"
url = base_url + "{}?appId={}&statsDataId={}&lvCat01={}&cdCat02={}&cdCat03={}&sectionHeaderFlg={}".format(get_type, appid, stats_data_id, lv_cat_01, cd_cat_02, cd_cat_03, section_header_flg)
data_all_age_die = urllib.request.urlopen(url).read().decode("utf8")

In [8]:
# 全国、年齢階級別の人口

get_type="getStatsData"
stats_data_id="0003014709"
url = base_url + "{}?appId={}&statsDataId={}".format(get_type, appid, stats_data_id)
#print(url)
get_type="getSimpleStatsData"
stats_data_id="0003014709"
cd_cat_01 = "000"
cd_cat_02 = "001"
cd_cat_03_from = "01001"
cd_cat_03_to = "01021"
cd_time = "2009000000"
section_header_flg="2"
url = base_url + "{}?appId={}&statsDataId={}&cdCat01={}&cdCat02={}&cdCat03From={}&cdCat03To={}&cdTime={}&sectionHeaderFlg={}".format(get_type, appid, stats_data_id, cd_cat_01, cd_cat_02, cd_cat_03_from, cd_cat_03_to, cd_time, section_header_flg)
data_all_age_pop = urllib.request.urlopen(url).read().decode("utf8")

In [9]:
# 都道府県別の自殺数、データフレーム化

dlines = data_pref_die.splitlines()[2:]
jiscode, cnt = [], []
for line in dlines:
    line2 = line.replace('"', "").split(",")
    jiscode_tmp = line2[2]
    jiscode_tmp = int(jiscode_tmp)-1
    if jiscode_tmp > 47:
        continue
    jiscode_tmp = "0" + str(jiscode_tmp) if jiscode_tmp < 10 else str(jiscode_tmp)
    jiscode.append(jiscode_tmp)
    cnt_tmp = int(line2[11])
    cnt.append(cnt_tmp)
df_pref_die = pd.DataFrame({"jiscode": jiscode, "cnt": cnt})
df_pref_die.head()

,jiscode,cnt
0,01,1439
1,02,476
2,03,459
3,04,577
4,05,416


In [10]:
# 都道府県コードマスタ、都道府県別、年齢階級別の人口、データフレーム化

dlines = data_pref_age_pop.splitlines()[2:]
jiscode, name, age_cls, cnt = [], [], [], []
for line in dlines:
    line2 = line.replace('"', "").split(",")
    jiscode_tmp = line2[6]
    jiscode_tmp = jiscode_tmp.replace("000", "")
    jiscode.append(jiscode_tmp)
    name_tmp = line2[7]
    name.append(name_tmp)
    age_cls_tmp = line2[5]
    age_cls_tmp = age_cls_tmp.replace("歳", "").split("～")
    age_cls_tmp = "85_" if len(age_cls_tmp) == 1 else age_cls_tmp[0]+"_"+age_cls_tmp[1]
    age_cls.append(age_cls_tmp)
    cnt_tmp = line2[11]
    cnt_tmp = int(cnt_tmp)*1000
    cnt.append(cnt_tmp)
df_pref_age_pop = pd.DataFrame({"jiscode": jiscode, "age_cls": age_cls, "cnt": cnt})
df_pref_age_pop_14 = df_pref_age_pop.query("age_cls=='0_4' | age_cls=='5_9' | age_cls=='10_14'")
df_pref_age_pop_14 = df_pref_age_pop_14[["jiscode", "cnt"]].groupby("jiscode").sum().reset_index()
col14 = pd.DataFrame([["_14"]*47]).T
col14.columns = ["age_cls"]
df_pref_age_pop_14 = pd.concat([df_pref_age_pop_14, col14], axis=1)
df_pref_age_pop = df_pref_age_pop.query("age_cls!='0_4' & age_cls!='5_9' & age_cls!='10_14'")
df_pref_age_pop = pd.concat([df_pref_age_pop, df_pref_age_pop_14], axis=0)
jiscode = sorted(set(jiscode), key=jiscode.index)
name = sorted(set(name), key=name.index)
df_pref_mst = pd.DataFrame({"jiscode": jiscode, "name": name})
df_pref_mst.head()

,jiscode,name
0,01,北海道
1,02,青森県
2,03,岩手県
3,04,宮城県
4,05,秋田県


In [11]:
# 全国の年齢階級別の自殺数、データフレーム化

dlines = data_all_age_die.splitlines()[2:]
age_cls, cnt = [], []
for line in dlines:
    line2 = line.replace('"', "").split(",")
    age_cls_tmp = line2[3]
    if age_cls_tmp == "不詳":
        continue
    age_cls_tmp = re.sub(r"歳|～", "", age_cls_tmp)
    age_cls_tmp = "100_" if age_cls_tmp == "100" else str(int(age_cls_tmp.split("-")[0]))+"_"+str(int(age_cls_tmp.split("-")[-1]))
    age_cls.append(age_cls_tmp)
    cnt_tmp = line2[-1]
    cnt_tmp = 0 if cnt_tmp == "-" else int(cnt_tmp)
    cnt.append(cnt_tmp)
age_cls = age_cls[3:-3]
age_cls.insert(0, "_14")
age_cls[-1] = "85_"
cnt_15 = np.sum(cnt[0:3])
cnt_85 = np.sum(cnt[-4:])
cnt = cnt[3:-4]
cnt.insert(0, cnt_15)
cnt.append(cnt_85)
df_all_age_die = pd.DataFrame({"age_cls" : age_cls, "cnt" : cnt})
df_all_age_die.head()

,age_cls,cnt
0,_14,55
1,15_19,457
2,20_24,1474
3,25_29,1739
4,30_34,2003


In [12]:
# 全国の年齢階級別の人口、データフレーム化

dlines = data_all_age_pop.splitlines()[2:]
age_cls, cnt = [], []
for line in dlines:
    line2 = line.replace('"', "").split(",")
    age_cls_tmp = line2[5]
    age_cls_tmp = re.sub(r"歳", "", age_cls_tmp).split("～")
    age_cls_tmp = "100_" if len(age_cls_tmp) == 1 else age_cls_tmp[0]+"_"+age_cls_tmp[1]
    age_cls.append(age_cls_tmp)
    cnt.append(int(line2[-1])*1000)
age_cls = age_cls[3:-3]
age_cls.insert(0, "_14")
age_cls[-1] = "85_"
cnt_15 = np.sum(cnt[0:3])
cnt_85 = np.sum(cnt[-4:])
cnt = cnt[3:-4]
cnt.insert(0, cnt_15)
cnt.append(cnt_85)
df_all_age_pop = pd.DataFrame({"age_cls" : age_cls, "cnt" : cnt})
df_all_age_pop.head()

,age_cls,cnt
0,_14,17010000
1,15_19,6079000
2,20_24,6913000
3,25_29,7502000
4,30_34,8591000


In [13]:
# 都道府県別、期待死亡数、データフレーム

df_y_tmp = pd.merge(df_all_age_pop, df_all_age_die, how="inner", on="age_cls", suffixes=("_pop", "_die"))
df_y_tmp = df_y_tmp.assign(y_tmp=df_y_tmp["cnt_die"]/df_y_tmp["cnt_pop"])
df_y_tmp = df_y_tmp.drop("cnt_pop", axis=1).drop("cnt_die", axis=1)
df_y_tmp = pd.merge(df_pref_age_pop, df_y_tmp, how="inner", on="age_cls")
df_y_tmp = df_y_tmp.assign(y=df_y_tmp["y_tmp"]*df_y_tmp["cnt"])
df_y_tmp = df_y_tmp.drop("cnt", axis=1).drop("y_tmp", axis=1)
df_y = df_y_tmp[["jiscode", "y"]].groupby("jiscode").sum().reset_index()
df_y.head()

,jiscode,y
0,01,1350.580604
1,02,336.687985
2,03,327.522655
3,04,557.454290
4,05,273.922115


In [14]:
# 都道府県別、隣接都道府県、データフレーム

from_names, to_names = [], []
with open("./data/japan_adj.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        from_name_tmp = row[1].replace("\xa0", "").replace(" ", "")
        to_names_tmp = row[4].replace("\xa0", "").replace(" ", "").split(",")
        for to_name_tmp in to_names_tmp:
            from_names.append(from_name_tmp)
            to_names.append(to_name_tmp)
from_names = from_names[0:-1]
to_names = to_names[0:-1]
df_adj_tmp = pd.DataFrame({"f_name": from_names, "t_name": to_names})
df_adj_tmp = pd.merge(df_adj_tmp, df_pref_mst, how="left", left_on="f_name", right_on="name").drop("name", axis=1).rename(columns={"jiscode": "f_jiscode"})
df_adj_tmp = pd.merge(df_adj_tmp, df_pref_mst, how="left", left_on="t_name", right_on="name").drop("name", axis=1).rename(columns={"jiscode": "t_jiscode"})
df_adj = df_adj_tmp.drop("f_name", axis=1).drop("t_name", axis=1)
df_adj.head()

,f_jiscode,t_jiscode
0,01,02
1,02,01
2,02,03
3,02,05
4,03,02


In [15]:
# 自殺数、期待死亡数、隣接行列 for 階層ベイズモデル

jiscodes = list(df_pref_mst["jiscode"].values.flatten())
n = len(jiscodes)
y_samples, z_samples, adj_matrix, d_matrix = [], [], [], []
for i in range(0, n):
    y = df_y[df_y["jiscode"] == jiscodes[i]]["y"].values[0] # 期待死亡数
    y_samples.append(y)
    z = df_pref_die[df_pref_die["jiscode"] == jiscodes[i]]["cnt"].values[0] # 自殺数
    z_samples.append(z)
    adjs = df_adj[df_adj["f_jiscode"] == jiscodes[i]]["t_jiscode"].values #  隣接JISCODE
    adjs_idx = np.zeros(n)
    for j in adjs:
        adjs_idx[jiscodes.index(j)] = 1
    adj_matrix.append(adjs_idx)
    d_idx = np.zeros(n)
    d_idx[i] = 2
    d_matrix.append(d_idx)
jiscodes = np.array(jiscodes, dtype=object) # JISCODE
y_samples = np.array(y_samples, dtype=int) # 期待死亡数
z_samples = np.array(z_samples, dtype=int) # 自殺数
adj_matrix = np.array(adj_matrix, dtype=int).reshape(n, n) # 隣接行列
d_matrix = np.array(d_matrix, dtype=int).reshape(n, n) # 自身の行列

In [19]:
# 空間構造のある階層ベイズモデルで推定する

with pm.Model() as model:
    
    p1 = pm.Uniform("p1", lower=0, upper=1)
    p2 = pm.Uniform("p2", lower=0, upper=1)
    tau_phi = pm.Gamma("tau_phi", alpha=0.5, beta=0.005)
    phi = pm.Normal("phi", mu=0, tau=tau_phi)
    t = p1*d_matrix+p2*adj_matrix
    psi = pm.MvNormal('psi', mu=0, cov=t, shape=n)
    lamb = np.exp(np.log(y_samples)+phi+psi)
    z = pm.Poisson("z", mu=lamb, observed=z_samples)
    start = pm.find_MAP()
    step = pm.NUTS()
    trace = pm.sample(1000, step, start=start)
    #pm.traceplot(trace)
    #pm.summary(trace)

logp = -inf, ||grad|| = 579.31: 100%|██████████| 4/4 [00:00<00:00, 356.60it/s]
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [psi, phi, tau_phi, p2, p1]
Sampling 2 chains:   0%|          | 0/3000 [00:00<?, ?draws/s]


RuntimeError: Chain 0 failed.

In [ ]:
phi_m = np.mean(trace["phi"])
psi_m = []
t = trace["psi"].transpose()
for i in t:
    psi_m.append(np.mean(i))
smr_est = np.exp(phi_m+psi_m)

# 都道府県別、推定SMR
df_smr_est = pd.DataFrame({"jiscode" : jiscodes, "smr_est" : smr_est})
display(df_smr_est)

In [ ]:
# 推定値を地図上に可視化する

location = [35.709634, 139.392101]
tiles="Stamen Toner"
zoom_start = 10
m = folium.Map(location=location, tiles=tiles, zoom_start=zoom_start)
m.choropleth(
    geo_path="./data/japan.geojson",
    data=df_smr_est,
    columns=["jiscode", "smr_est"],
    key_on="properties.JCODE",
    threshold_scale=[0.90, 0.95, 1.00, 1.05, 1.10, 1.15],
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    reset=True)
display(m)